# Análise de Sentimento - Bag of Words Meets Bag of Popcorn

O presente projeto foi realizado para a disciplina Programação para Aprendizagem de Máquina, da pós graduação em Ciência de Dados da Universidade Federal de Pernambuco.

Foi utilizada a base de dados da competição do Kaggle "Bag of words meet bags of popcorns", que contém 50.000 comentários a respeito de filmes no site do IMDb. Os comentários são divididos em 2 grupos: os que foram acompanhados de notas inferiores a 5 estrelas -rotulados como 0- são de comentários com sentimento negativo a respeito do filme, os que foram acompanhados de notas superiores a 7 -rotulados com 1- são comentários com sentimento positivo.

O objetivo do projeto criar um modelo para prever o sentimento do comentário.

## Preparando o ambiente

In [1]:
# Código para o Colab

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

%cd /content/gdrive/MyDrive/

Mounted at /content/gdrive
/content/gdrive/MyDrive


In [2]:
#Importando bibliotecas relevantes

import pandas as pd #Biblioteca para visualização e manipulação de dataframes
import numpy as np #Biblioteca com métodos matemáticos como geração aleatória de números, algebra linear, etc

#Bibliotecas gráficas para visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

import nltk #Biblioteca para processamento simbólico e estatístico de linguagem natural
from nltk.corpus import stopwords

from tqdm import tqdm #Biblioteca para visualização do processamos de uma iteração

In [17]:
#Importando base de dados
train = pd.read_csv('labeledTrainData.tsv',sep='\t')

train.review = [x.replace(".","").replace(",","").replace("!","").
                replace("(","").replace(")","").replace("?","").
                replace("<br","").replace("<","").replace("\t","")
                .replace(">","").replace("...","").replace("\\","").replace("\"","") for x in train.review]

train.review

test = pd.read_csv('testData.tsv',sep='\t')
test.review = [x.replace(".","").replace(",","").replace("!","").
                replace("(","").replace(")","").replace("?","").
                replace("<br","").replace("<","").replace("\t","")
                .replace(">","").replace("...","").replace("\\","").replace("\"","") for x in test.review]



In [7]:
train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [8]:
train.shape

(25000, 3)

## Metodologia

O modelo selecionado para o projeto foi o Naive Bayes Classifier. Esse classificador usa a regra de Bayes para prever o output (sentimento positivo ou negativo) do modelo, dadas as informações do review, conforme descrito abaixo:
<br />

$       P(sentimento | review) = \frac{P(review | sentimento)*P(sentimento)}{P(review)}$

<br />
Em resumo, será contado o uso de palavras nos reviews resultados em cada sentimento e determinada uma probabilidade de cada palavra estar relacionada a um sentimento negativo ou positivo.

É importante destacar que o modelo não considera a interação entre palavras nem a ordem de aparecimento das mesmas.

In [7]:
stop_words = set(stopwords.words('english'))
words = []
for n in tqdm([s for s in train.review.str.split(' ')]):
    for w in n:
        if w not in stop_words:
            words.append(w)
        else:
            pass
all_words = nltk.FreqDist(words)
word_features = word_features[:2000]

100%|██████████| 25000/25000 [00:01<00:00, 16315.92it/s]


In [13]:
documents = []
for i in tqdm(range(len(train))):
    l=[]
    for s in train.review[i].split(' '):
        l.append(s)
    documents.append((l,train.sentiment[i]))

100%|██████████| 25000/25000 [00:02<00:00, 9269.83it/s]


In [10]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [15]:
featuresets = [(document_features(d), c) for (d,c) in documents]

In [18]:
train_set, test_set = featuresets[100:], featuresets[:100]

In [19]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [20]:
print(nltk.classify.accuracy(classifier, test_set))

0.79


In [22]:
classifier.show_most_informative_features(10)

Most Informative Features
       contains(unfunny) = True                0 : 1      =     15.7 : 1.0
         contains(waste) = True                0 : 1      =     13.5 : 1.0
     contains(redeeming) = True                0 : 1      =     10.0 : 1.0
     contains(pointless) = True                0 : 1      =     10.0 : 1.0
         contains(worst) = True                0 : 1      =      9.0 : 1.0
     contains(laughable) = True                0 : 1      =      8.6 : 1.0
         contains(awful) = True                0 : 1      =      8.5 : 1.0
        contains(poorly) = True                0 : 1      =      8.2 : 1.0
   contains(wonderfully) = True                1 : 0      =      7.8 : 1.0
         contains(sucks) = True                0 : 1      =      6.9 : 1.0
